## MISA (2024-2025)
- Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

- Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
chatgpt youtube


---

# Linear Regression

In [ ]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import LinearRegression as LR
from sklearn.linear_model import Ridge 

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target
print(X_train)

### Inverse

In [ ]:
def fit_inverse(X, y):
    """Direct method using the inverse (without bias term)"""
    
    # Calculer la matrice X^T X
    XtX = np.dot(X.T,X)

    # Vérifier si X^T X est inversible (si son déterminant est non nul)
    if np.linalg.det(XtX) == 0:
        raise ValueError("La matrice X^T X est singulière, donc l'inverse ne peut pas être calculé.")
    
    # Calculer l'inverse de X^T X
    XtX_inv = np.linalg.inv(XtX)

    # Calculer les coefficients beta
    Xt_y = np.dot(X.T, y)

    beta = np.dot(XtX_inv, Xt_y)
    
    return beta

In [16]:
w = fit_inverse(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-12

[ -10.0098663  -239.81564367  519.84592005  324.3846455  -792.17563855
  476.73902101  101.04326794  177.06323767  751.27369956   67.62669218]
prediction error:  4.667213674682389e-14


### Cholesky

In [4]:
def fit_cholesky(X, y):
    """ Cholesky approach """
    # Calculer X^T X
    XtX = np.dot(X.T, X)

    # Vérifier que la matrice XtX est définie positive avant de faire la décomposition de Cholesky
    if np.all(np.linalg.eigvals(XtX) > 0):
        # Calculer la décomposition de Cholesky de XtX
        L = np.linalg.cholesky(XtX)

        # Résoudre Lz = X^T y pour z (première étape)
        z = np.linalg.solve(L, np.dot(X.T, y))

        # Résoudre L^T beta = z pour beta (deuxième étape)
        beta = np.linalg.solve(L.T, z)
        return beta
    else:
        raise ValueError("La matrice X^T X n'est pas définie positive, décomposition de Cholesky échouée.")


In [5]:
w = fit_cholesky(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-12

prediction error:  2.848242565728278e-14


### QR

In [6]:
def fit_qr(X, y):
    """ QR approach """
    # Décomposition QR de X
    Q, R = np.linalg.qr(X)
    
    # Résoudre pour beta en utilisant R et Q^T y
    Qt_y = np.dot(Q.T, y)  # Calculer Q^T y
    beta = np.linalg.solve(R, Qt_y)  # Résoudre R beta = Q^T y
    
    return beta

In [7]:
w = fit_qr(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-12

prediction error:  1.8722092877528735e-14


### SVD

In [8]:
def fit_svd(X, y):
    """ SVD approach for solving X^T X beta = X^T y """
    
    # Calculer la décomposition SVD de X
    U, Sigma, Vt = np.linalg.svd(X, full_matrices=False)
    
    # Calculer X^T y
    Xt_y = np.dot(X.T, y)  # Ce produit est de taille (n,)
    
    # Inverser Sigma (les valeurs singulières de X)
    Sigma_inv = np.diag(1.0 / Sigma)  # Inverser les valeurs singulières
    
    # Calculer beta = V * Sigma_inv * U.T * y
    # On peut directement calculer la multiplication dans l'ordre suivant :
    beta = np.dot(Vt.T, np.dot(Sigma_inv, np.dot(U.T, y)))
    
    return beta

In [9]:
w = fit_svd(X_train, y_train)

sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)

error = rel_error(sk_model.coef_, w)
print("prediction error: ", error)
assert error <= 1e-12

prediction error:  1.650383542758461e-14


### Combine everything in a class

In [10]:
class LinearRegression():
    def __init__(self, fit_intercept=True, method="inverse"):
        self.w = 0
        self.fit_intercept = fit_intercept  # inclure un biais (intercept) ou non
        self.method = method  # méthode pour ajuster le modèle: 'inverse', 'cholesky', 'qr', 'svd'

    def fit(self, X, y):
        """
        Ajuste le modèle aux données en utilisant la méthode spécifiée (inverse, cholesky, qr, svd).
        """
        # Si on souhaite ajuster un biais (intercept), ajouter une colonne de 1 à X
        if self.fit_intercept:
            X = np.column_stack([np.ones(X.shape[0]), X])  # Ajouter une colonne de 1 pour le biais
        
        # Utiliser la méthode spécifiée pour calculer les coefficients
        if self.method == "inverse":
            self.w = fit_inverse(X, y)  # Appel à la fonction fit_inverse
        elif self.method == "cholesky":
            self.w = fit_cholesky(X, y)  # Appel à la fonction fit_cholesky
        elif self.method == "qr":
            self.w = fit_qr(X, y)  # Appel à la fonction fit_qr
        elif self.method == "svd":
            self.w = fit_svd(X, y)  # Appel à la fonction fit_svd
        else:
            raise ValueError("Méthode spécifiée invalide. Choisissez 'inverse', 'cholesky', 'qr', ou 'svd'.")

    def predict(self, X):
        """
        Prédire les valeurs de y pour les nouvelles données X.
        """
        # Si on a un biais, ajouter une colonne de 1 à X
        if self.fit_intercept:
            X = np.column_stack([np.ones(X.shape[0]), X])  # Ajouter une colonne de 1 pour le biais
        
        # Prédictions en utilisant X et les coefficients calculés
        return np.dot(X, self.w)


#### without the bias term

In [11]:
# DIRECT INVERSE APPROACH
sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error: ", error)
assert error <= 1e-11

prediction error:  4.216526906620261e-13


In [12]:
# OTHER APPROACHES
sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model_cholesky = LinearRegression(fit_intercept=False, method="cholesky")
model_cholesky.fit(X_train, y_train)
pred_cholesky = model_cholesky.predict(X_train)

error_cholesky = rel_error(pred_cholesky, sk_pred)
print("prediction error cholesky: ", error_cholesky)
assert error_cholesky <= 1e-11

model_qr = LinearRegression(fit_intercept=False, method="qr")
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
print("prediction error qr: ", error_qr)
assert error_qr <= 1e-11

model_svd = LinearRegression(fit_intercept=False, method="svd")
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
print("prediction error svd: ", error_svd)
assert error_svd <= 1e-11

prediction error cholesky:  2.2400299191415709e-13
prediction error qr:  1.778847288729989e-13
prediction error svd:  2.5694460837212984e-13


#### with the bias term

In [13]:
# DIRECT INVERSE APPROACH
sk_model = LR(fit_intercept=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error: ", error)
assert error <= 1e-13

prediction error:  4.662859025350709e-15


In [14]:
# OTHER APPROACHES
sk_model = LR(fit_intercept=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model_cholesky = LinearRegression(fit_intercept=True, method="cholesky")
model_cholesky.fit(X_train, y_train)
pred_cholesky = model_cholesky.predict(X_train)

error_cholesky = rel_error(pred_cholesky, sk_pred)
print("prediction error cholesky: ", error_cholesky)
assert error_cholesky <= 1e-11

model_qr = LinearRegression(fit_intercept=True, method="qr")
model_qr.fit(X_train, y_train)
pred_qr = model_qr.predict(X_train)

error_qr = rel_error(pred_qr, sk_pred)
print("prediction error qr: ", error_qr)
assert error_qr <= 1e-11

model_svd = LinearRegression(fit_intercept=True, method="svd")
model_svd.fit(X_train, y_train)
pred_svd = model_svd.predict(X_train)

error_svd = rel_error(pred_svd, sk_pred)
print("prediction error svd: ", error_svd)
assert error_svd <= 1e-11

prediction error cholesky:  1.0937132350428517e-15
prediction error qr:  1.4921148881122223e-15
prediction error svd:  3.1707441372384776e-15
